# Training of the model for PID

## Generate random data for training

### Import libraries

In [1]:
import numpy as np
from typing import List

# Local imports
from PID import PID
from Systems import Trolley
from simulation import SimulationInfo

In [4]:
DT = 0.01
TIME = np.arange(0, 50, DT)
TARGET = np.ones(len(TIME)) * 5
DISTRUBANCE = np.zeros(len(TIME))


def run_simulation(kp, ki, kd) -> List[SimulationInfo]:
    # Create PID controller
    pid = PID(kp, ki, kd)

    # Create system
    system = Trolley(mass=2, friction=0.1, dt=DT)

    simInfo: List[SimulationInfo] = []
    for i in range(1, len(TIME)):
        time = TIME[i]
        target = TARGET[i]
        distrubance = DISTRUBANCE[i]
        position = system.get_position()

        # Compute the control output
        control_outpot = pid.compute(target, position, DT)
        

        # Update the simulation object
        system.update(control_outpot)
        U, dUdt, dU2dt2 = system.get_U()

        # Save
        simInfo.append(
            SimulationInfo(
                time=time,
                target=target,
                disturbance=distrubance,
                U=U,
                dUdt=dUdt,
                dU2dt2=dU2dt2
            )
        )
    return simInfo



def generate_training_data(num_simples: int) -> List[List[SimulationInfo]]:
    # Generate random PID parameters
    KP = np.random.uniform(low=0, high=5, size=num_simples)
    KI = np.random.uniform(low=0, high=5, size=num_simples)
    KD = np.random.uniform(low=0, high=5, size=num_simples)

    K = np.array([KP, KI, KD]).T

    simulations: List[List[SimulationInfo]] = []
    for kp, ki, kd in K:
        simulations.append(run_simulation(kp, ki, kd))

    return simulations


simulations = generate_training_data(100)


## Start training of the model

In [7]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM

### Create the model

In [12]:
# Set seed
np.random.seed(1000)

# Create model
model = Sequential()
model.add(Dense(units=12, activation='relu', input_dim=6))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=3, activation='relu'))

# Compile model
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


# Create a custom loss function
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

### Make training and validation data

In [11]:
# Compile model
model.compile(optimizer=optimizer, loss=custom_loss)

